<a href="https://colab.research.google.com/github/javierarteagapuell/deeplearning/blob/main/Prac2Ej1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LIBRERIAS mlxtend o apyori

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

from mlxtend.frequent_patterns import apriori, association_rules

#pip install mlxtend  

In [3]:
data = pd.read_csv('BlackFriday.csv', encoding='UTF-8')
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [4]:
data.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3'],
      dtype='object')

In [5]:
data.isnull().values.any() 

True

Vemos en que columnas hay NA's

In [6]:
data.isna().sum()  

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72344
Product_Category_3            162562
dtype: int64

Como más de la mitad de las filas de Product Category 3 son NA's eliminamos esta columna

In [7]:
data = data.drop(['Product_Category_3'], axis=1)

Vemos si hay algún dato duplicado

In [8]:
if any(data.duplicated()):
  print("Hay datos duplicados")
else:
  print("No hay datos duplicados")

No hay datos duplicados


- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.

In [9]:
data.loc[(data['Gender'] == 'M'),'Male'] = True
data.loc[(data['Gender'] == 'F'),'Male'] = False
data.loc[(data['Gender'] == 'F'),'Female'] = True
data.loc[(data['Gender'] == 'M'),'Female'] = False

data.loc[(data['Age'] != '0-17'), 'Menores'] = False
data.loc[(data['Age'] == '0-17'), 'Menores'] = True
data.loc[(data['Age'] != '18-25'), 'Uni'] = False
data.loc[(data['Age'] == '18-25'), 'Uni'] = True
data.loc[(data['Age'] != '26-35'), 'PostUni'] = False
data.loc[(data['Age'] == '26-35'), 'PostUni'] = True
data.loc[(data['Age'] != '36-45'), 'Medio'] = False
data.loc[(data['Age'] == '36-45'), 'Medio'] = True
data.loc[(data['Age'] != '46-50'), 'Mayor'] = False
data.loc[(data['Age'] == '46-50'), 'Mayor'] = True
data.loc[(data['Age'] != '50-55'), 'PreJubi'] = False
data.loc[(data['Age'] == '50-55'), 'PreJubi'] = True
data.loc[(data['Age'] != '55+'), 'Jubi'] = False
data.loc[(data['Age'] == '55+'), 'Jubi'] = True



data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Male,Female,Menores,Uni,PostUni,Medio,Mayor,PreJubi,Jubi
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,True,False,False,False,False,False,True,False,False
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,True,False,False,False,True,False,False,False,False
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,False,True,False,False,False,True,False,False,False
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,False,True,False,False,False,True,False,False,False
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,False,True,False,False,True,False,False,False,False


Colocamos los datos de la forma indicada para poder calcular las frecuencias de estos más adelante

In [10]:
data1 = pd.DataFrame(data,columns=['User_ID','Male','Female','Menores','Uni','PostUni','Medio','Mayor','PreJubi','Jubi']).set_index('User_ID')
data1.head()

,Male,Female,Menores,Uni,PostUni,Medio,Mayor,PreJubi,Jubi
User_ID,,,,,,,,,
1000004,True,False,False,False,False,False,True,False,False
1000009,True,False,False,False,True,False,False,False,False
1000010,False,True,False,False,False,True,False,False,False
1000010,False,True,False,False,False,True,False,False,False
1000011,False,True,False,False,True,False,False,False,False


In [11]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(data1, min_support=0.06, use_colnames=True)
frequent_itemsets_sort = frequent_itemsets.sort_values(by=['support'], ascending=False)

frequent=frequent_itemsets_sort.support*len(data1)
frequent_itemsets_sort['frecuencia'] =frequent
frequent_itemsets_sort.head()

,support,itemsets,frecuencia
0,0.752452,(Male),175772.0
3,0.39995,(PostUni),93428.0
7,0.30715,"(Male, PostUni)",71750.0
1,0.247548,(Female),57827.0
4,0.199962,(Medio),46711.0


In [12]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.6)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Uni),(Male),0.181050,0.752452,0.136362,0.753174,1.000960,0.000131,1.002927
1,(PostUni),(Male),0.399950,0.752452,0.307150,0.767971,1.020625,0.006207,1.066885
2,(Medio),(Male),0.199962,0.752452,0.149915,0.749716,0.996365,-0.000547,0.989070
3,(Mayor),(Male),0.083806,0.752452,0.060064,0.716708,0.952497,-0.002996,0.873828


In [13]:
import inspect

rules_sort = rules.sort_values(by=['confidence'], ascending=False)
rules_sort

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(PostUni),(Male),0.399950,0.752452,0.307150,0.767971,1.020625,0.006207,1.066885
0,(Uni),(Male),0.181050,0.752452,0.136362,0.753174,1.000960,0.000131,1.002927
2,(Medio),(Male),0.199962,0.752452,0.149915,0.749716,0.996365,-0.000547,0.989070
3,(Mayor),(Male),0.083806,0.752452,0.060064,0.716708,0.952497,-0.002996,0.873828


In [14]:
rules_maximales = rules [max(rules)]
rules_maximales

0    0.136362
1    0.307150
2    0.149915
3    0.060064
Name: support, dtype: float64

In [15]:
any(rules.duplicated())

False

In [16]:
plt(rules)

TypeError: ignored

In [ ]:
plt(rules, method='graph', 'interactive'=True, shading=NA)

Agrupamos los productos por usuario

In [ ]:
Tabla_Usuario_Producto = data.groupby('User_ID')['Product_ID'].apply(','.join)
Tabla_Usuario_Producto

Pasamos los productos a forma boleana

Si el usuario ha comprado ese producto True
sino False

In [ ]:
Server_ID = data.groupby(['User_ID','Product_ID'])['Occupation'].sum().apply(lambda x: x>0).unstack().reset_index().fillna(False).set_index('User_ID')
Server_ID

Itemset frecuentes para K==1 (Son los que tengan un soporte alto)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(Server_ID, min_support=0.06, use_colnames=True)
frequent_itemsets_sort = frequent_itemsets.sort_values(by=['support'], ascending=False)

frequent=frequent_itemsets_sort.support*len(Server_ID)
frequent_itemsets_sort['frecuencia'] =frequent
frequent_itemsets_sort.head()

In [ ]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules.head()

2) Para k≥2.

a. Mostrar los itemset frecuentes candidatos y su soporte.

In [ ]:
Server_ID2 = data.groupby(['User_ID','Product_ID'])['Occupation'].sum().apply(lambda x: x>=2).unstack().reset_index().fillna(False).set_index('User_ID')
Server_ID2.head()

3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes. CON 8 se para

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(Server_ID2, min_support=0.06, use_colnames=True)
frequent_itemsets.head()

In [ ]:
#Server_ID3 = data.groupby(['User_ID','Product_ID'])['Occupation'].sum().apply(lambda x: x>7).unstack().reset_index().fillna(False).set_index('User_ID')
#frequent_itemsets = apriori(Server_ID3, min_support=0.06, use_colnames=True)
#frequent_itemsets.head()

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

In [ ]:
#rules = association_rules(frq_items, metric ="lift", min_threshold = 0.8)
#rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules.head()

5) Listar todas las reglas que sean de alta confianza.

In [ ]:
import inspect

rules_sort = rules.sort_values(by=['confidence'], ascending=False)
rules_sort

A partir de lo anterior se pide:

- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
(1 punto)

- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos

-   Para   una   de   las   configuraciones,   interpreta   algunas   de   las   reglas   que   te   hayan   resultado   interesantes
obtenidas   usando   la   clase  association_rules  y   un   par   de   configuraciones   cambiando   la   variable  metric  y
min_threshold. Justifica los resultados. (1 punto)

In [ ]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)#, target = "rules")
rules.head()

- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar  de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)

In [ ]:
import inspect

inspect(sorted(x = rules, decreasing = True, by = 'confidence'))

In [ ]:
#reglas_maximales <- reglas [is.maximal(reglas)]
rules_maximales = rules [max(rules)]
rules_maximales

In [ ]:
#reglas_redundantes <- reglas[is.redundant(x = reglas, measure = "confidence")]

rules.duplicated(keep=False)

In [ ]:
plt(rules)

In [ ]:
plt(rules, method='graph', 'interactive'=True, shading=NA)